In [ ]:
import os

# ------------------------------------------------------------
# API Key Setup 
# ------------------------------------------------------------
# Do NOT hard-code your API key in this notebook.
# Make sure to set your GOOGLE_API_KEY as an environment variable
# before running this notebook:
#
# Mac/Linux:
#   export GOOGLE_API_KEY="your_key_here"
#
# Windows (PowerShell):
#   setx GOOGLE_API_KEY "your_key_here"
#
# In cloud environments, add it through project environment settings.
# ------------------------------------------------------------

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError(
        "GOOGLE_API_KEY not found. Please set it as an environment variable before running."
    )


In [ ]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from google.adk.agents import LlmAgent

print("✅ ADK components imported successfully.")

In [ ]:
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, field
from datetime import datetime, timedelta
import time
import json
import uuid
from enum import Enum
import pandas as pd

print("✅ Additional libraries imported for evaluation")

In [ ]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

## EVALUATION FRAMEWORK ##

In [ ]:
#Add Evaluation Framework

@dataclass
class QuickMetrics:
    """Simplified metrics for quick evaluation"""
    agent_name: str
    execution_time: float = 0.0
    success: bool = False
    confidence_score: float = 0.0
    details: Dict = field(default_factory=dict)

    def to_dict(self):
        return {
            'agent_name': self.agent_name,
            'execution_time': self.execution_time,
            'success': self.success,
            'confidence_score': self.confidence_score,
            'details': self.details
        }

class QuickEvaluator:
    """Lightweight evaluator for notebook use"""
    
    def __init__(self):
        self.metrics: List[QuickMetrics] = []
    
    def evaluate(self, agent_name: str, success: bool, 
                confidence: float, details: Dict = None) -> QuickMetrics:
        """Quick evaluation"""
        metric = QuickMetrics(
            agent_name=agent_name,
            success=success,
            confidence_score=float(confidence),
            details=details or {}
        )
        self.metrics.append(metric)
        return metric
    
    def get_summary(self) -> Dict:
        """Get evaluation summary"""
        if not self.metrics:
            return {
                'total_agents': 0,
                'success_rate': 0.0,
                'avg_confidence': 0.0,
                'failed_agents': []
            }
        
        return {
            'total_agents': len(self.metrics),
            'success_rate': sum(1 for m in self.metrics if m.success) / len(self.metrics),
            'avg_confidence': sum(m.confidence_score for m in self.metrics) / len(self.metrics),
            'failed_agents': [m.agent_name for m in self.metrics if not m.success]
        }
    
    def print_summary(self):
        """Print readable summary"""
        summary = self.get_summary()
        print("\n" + "="*60)
        print("EVALUATION SUMMARY")
        print("="*60)
        print(f"Total Agents Evaluated: {summary.get('total_agents', 0)}")
        print(f"Success Rate: {summary.get('success_rate', 0)*100:.1f}%")
        print(f"Average Confidence: {summary.get('avg_confidence', 0):.2f}")
        if summary.get('failed_agents'):
            print(f"Failed Agents: {', '.join(summary['failed_agents'])}")
        print("="*60 + "\n")

# Initialize evaluator
evaluator = QuickEvaluator()
print("✅ Quick evaluator initialized")

## DATA AGENT 

In [ ]:
import yfinance as yf

class DataAgent:
    def __init__(self, name, model, instruction, output_key, evaluator):
        self.name = name
        self.model = model
        self.instruction = instruction
        self.output_key = output_key
        self.evaluator = evaluator

    def fetch_stock_timeseries(self, ticker, start_date, end_date, interval='1d'):
        """
        Fetch historical stock price data from Yahoo Finance.
        
        Args:
            ticker (str): Stock symbol (e.g., 'AAPL').
            start_date (str): Start date in 'YYYY-MM-DD' format.
            end_date (str): End date in 'YYYY-MM-DD' format.
            interval (str): Data interval (e.g., '1d', '1wk', '1mo').
        
        Returns:
            pd.DataFrame: Time series data with date as index.
        """
        start_time = time.time()
        try:
            
            stock = yf.Ticker(ticker)
            hist = stock.history(start=start_date, end=end_date, interval=interval)
            # Ensure the Date is a column if you want it (could also be index)
            hist.reset_index(inplace=True)

            #Evaluate data quality
            data_complete = len(hist) > 0 and 'Close' in hist.columns
            if data_complete:
                # Check for missing values in critical columns
                critical_cols = ['Close', 'Volume']
                available_cols = [col for col in critical_cols if col in hist.columns]
                
                if available_cols:
                    total_values = len(hist) * len(available_cols)
                    missing_values = hist[available_cols].isna().sum().sum()
                    data_accuracy = 1.0 - (missing_values / total_values) if total_values > 0 else 0.0
                else:
                    data_accuracy = 0.0
                
                confidence = (1.0 + data_accuracy) / 2.0  # Average of completeness and accuracy
            else:
                data_accuracy = 0.0
                confidence = 0.0

            execution_time = time.time() - start_time

            self.evaluator.evaluate(
                agent_name=self.name,
                success=data_complete,
                confidence=confidence,
                details={
                    'ticker': ticker,
                    'rows_fetched': len(hist),
                    'execution_time': execution_time,
                    'data_accuracy': data_accuracy,
                    'has_critical_columns': all(col in hist.columns for col in ['Close', 'Volume'])
                }
            )
        
            
            if data_complete:
                print(f"✅ {self.name}: Successfully fetched {len(hist)} rows in {execution_time:.2f}s")
                print(f"   Confidence Score: {confidence:.2f}")
            else:
                print(f"⚠️  {self.name}: Fetched incomplete data")

            return hist
        
        except Exception as e:
            execution_time = time.time() - start_time
            self.evaluator.evaluate(
                agent_name=self.name,
                success=False,
                confidence=0.0,
                details={'ticker': ticker,'error': str(e), 'execution_time': execution_time}
            )
            print(f"❌ {self.name} failed: {e}")
            print(f"   Execution time: {execution_time:.2f}s")
            return pd.DataFrame()
            
        

# Usage Example
data_agent = DataAgent(
    name="DataAgent",
    model="gemini-2.5-flash-lite",
    instruction="You are a specialized data agent fetching Yahoo Finance historical trend data.",
    output_key="data_findings",
    evaluator = evaluator
)

# Example: Pull Apple's stock data for the past year
timeseries_data = data_agent.fetch_stock_timeseries(
    ticker='AAPL', 
    start_date='2024-11-01', 
    end_date='2025-11-01', 
    interval='1d'
)

print("✅ data_agent created.")
# print(timeseries_data.head())

## ANALYSIS AGENT

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

class AnalysisAgent:
    def __init__(self, name, model, instruction, output_key, evaluator):
        self.name = name
        self.model = model
        self.instruction = instruction
        self.output_key = output_key
        self.evaluator = evaluator

    def analyze_trend(self, timeseries_df, ticker, value_col='Close', window=20):
        start_time = time.time()
        df = timeseries_df.copy()

        try:
            # Basic checks
            if df.empty or value_col not in df.columns:
                execution_time = time.time() - start_time
                self.evaluator.evaluate(
                    agent_name=self.name,
                   success=False,
                    confidence=0.0,
                    details={
                        'ticker': ticker,
                        'reason': 'Empty dataframe or missing value column',
                        'execution_time': execution_time
                    }
                )
                return {
                    'error': 'No data available for analysis',
                    'ticker': ticker
                }

            # Sort by date if present
            if 'Date' in df.columns:
                df = df.sort_values('Date')

            # --- Moving average ---
            df['MA'] = df[value_col].rolling(window).mean()

            latest_value = df[value_col].iloc[-1]
            latest_ma = df['MA'].iloc[-1]

            # Trend direction 
            if latest_value > latest_ma:
                trend_direction = 'upward'
            elif latest_value < latest_ma:
                trend_direction = 'downward'
            else:
                trend_direction = 'neutral'

            # Total & % change 
            first_value = df[value_col].iloc[0]
            change = latest_value - first_value
            pct_change = (change / first_value) * 100 if first_value != 0 else np.nan

            # --- Volatility metrics ---
            # Returns-based volatility
            df['Returns'] = df[value_col].pct_change()
            returns_volatility = float(df['Returns'].std()) if df['Returns'].notna().sum() > 1 else None

            # Std dev + Coefficient of Variation (CV)
            std_dev = float(df[value_col].std())
            mean_val = float(df[value_col].mean()) if df[value_col].mean() != 0 else None
            coefficient_variation = (std_dev / mean_val) if (mean_val is not None) else None

            if coefficient_variation is None:
                stability = "Unknown"
            elif coefficient_variation < 0.1:
                stability = "Stable"
            elif coefficient_variation < 0.3:
                stability = "Variable"
            else:
                stability = "Highly Variable"

            # --- Anomaly detection via z-scores ---
            anomalies = []
            anomaly_count = 0
            if std_dev > 0:
                z_scores = (df[value_col] - df[value_col].mean()) / std_dev
                df['z_score'] = z_scores
                anomaly_mask = z_scores.abs() > 2.5
                anomaly_df = df[anomaly_mask]
                anomaly_count = int(anomaly_df.shape[0])

                if anomaly_count > 0:
                    cols_to_keep = [value_col, 'z_score']
                    if 'Date' in df.columns:
                        cols_to_keep.insert(0, 'Date')
                    anomalies = anomaly_df[cols_to_keep].head(5).to_dict('records')

            # --- Trend summary ---
            trend_summary = {
                'ticker': ticker,
                'latest_value': float(latest_value),
                'latest_moving_average': float(latest_ma),
                'trend_direction': trend_direction,
                'total_change': float(change),
                'percentage_change': float(pct_change) if pct_change is not None else None,
                'moving_average_window': window,
                'returns_volatility_std': returns_volatility,
                'value_std_dev': std_dev,
                'coefficient_of_variation': coefficient_variation,
                'stability_label': stability,
                'anomaly_count': anomaly_count,
                'sample_anomalies': anomalies
            }

            # --- Confidence score (extended with risk) ---
            trend_strength = 1.0 if trend_direction in ['upward', 'downward'] else 0.6
            data_quality = 1.0 if len(df) > window * 2 else 0.6

            if stability == "Stable":
                volatility_quality = 1.0
            elif stability == "Variable":
                volatility_quality = 0.8
            elif stability == "Highly Variable":
                volatility_quality = 0.6
            else:
                volatility_quality = 0.7

            confidence = (
                0.5 * trend_strength +
                0.3 * data_quality +
                0.2 * volatility_quality
            )

            execution_time = time.time() - start_time

            # Observability
            self.evaluator.evaluate(
                agent_name=self.name,
                success=True,
                confidence=confidence,
                details={
                    'ticker': ticker,
                    'data_points_used': len(df),
                    'metrics_calculated': list(trend_summary.keys()),
                    'execution_time': execution_time,
                    'stability_label': stability,
                    'anomaly_count': anomaly_count
                }
            )

            print(f"[AnalysisAgent] {ticker} | Trend: {trend_direction}, "
                  f"Change: {pct_change:.2f}%, "
                  f"Stability: {stability}, "
                  f"Anomalies: {anomaly_count}, "
                  f"Confidence: {confidence:.2f}, "
                  f"Time: {execution_time:.3f}s")

            # Optional: plot the results
            if 'Date' in df.columns:
                plt.figure(figsize=(10, 5))
                plt.plot(df['Date'], df[value_col], label='Price')
                plt.plot(df['Date'], df['MA'], label=f'{window}-day MA')
                plt.xlabel('Date')
                plt.ylabel(value_col)
                plt.title('Stock Price Trend Analysis')
                plt.legend()
                plt.tight_layout()
                plt.show()

            return trend_summary

        except Exception as e:
            execution_time = time.time() - start_time
            self.evaluator.evaluate(
                agent_name=self.name,
                success=False,
                confidence=0.0,
                details={
                    'ticker': ticker,
                    'error': str(e),
                    'execution_time': execution_time
                }
            )
            print(f"[AnalysisAgent] ❌ Failed for {ticker}: {str(e)}")
            return {
                'error': 'Analysis failed',
                'ticker': ticker,
                'exception': str(e)
            }

# Usage Example
analysis_agent = AnalysisAgent(
    name="AnalysisAgent",
    model="gemini-2.5-flash-lite",
    instruction="You analyze the time series data and provide trend analysis.",
    output_key="trend_analysis",
    evaluator=evaluator
)

trend_results = analysis_agent.analyze_trend(
    timeseries_data,
    ticker='AAPL',
    value_col='Close',
    window=20
)

print(trend_results)

# print("✅ summarizer_agent created.")

In [ ]:
pip install feedparser

## INSIGHT AGENT

In [ ]:
import requests
import feedparser
import time
from datetime import datetime

class InsightAgent:
    def __init__(self, name, model, instruction, tools, output_key, evaluator):
        self.name = name
        self.model = model
        self.instruction = instruction
        self.tools = tools
        self.output_key = output_key
        self.evaluator = evaluator  # ✅ Added evaluator
    
    def get_google_news(self, company, max_results=5):
        """
        Fetches recent news headlines for a given company using Google News RSS.
        """
        try:
            url = f'https://news.google.com/rss/search?q={company}+stock'
            feed = feedparser.parse(url)
            news = []
            for entry in feed.entries[:max_results]:
                news.append({
                    'title': entry.title,
                    'link': entry.link,
                    'published': entry.published
                })
            return news
        except Exception as e:
            print(f"⚠️ Error fetching news: {e}")
            return []    
            
    def generate_insight(self, trend_summary, news_list, ticker=""):
        """
        Combines trend analysis and news for high-level insight and recommendation.
        NOW WITH DYNAMIC CONFIDENCE EVALUATION + RISK/STABILITY.
        """
        start_time = time.time()
        
        try:
            # ========================================
            # TRACK ANALYSIS QUALITY FOR CONFIDENCE
            # ========================================
            confidence_factors = []
            analysis_components = {
                'has_trend_data': False,
                'has_news_data': False,
                'trend_quality': 0.0,
                'news_quality': 0.0,
                'recommendation_confidence': 0.0
            }
            
            insights = []
            
            # ========================================
            # 1. TREND ANALYSIS
            # ========================================
            trend_comment = ""
            recommendation = "Hold"  # Default
            
            # Check if trend data is valid
            if trend_summary and isinstance(trend_summary, dict):
                analysis_components['has_trend_data'] = True
                
                trend_direction = trend_summary.get('trend_direction', 'unknown')
                pct_change = trend_summary.get('percentage_change', 0)
                
                # Validate trend data quality
                if trend_direction != 'unknown' and pct_change is not None:
                    analysis_components['trend_quality'] = 1.0
                    confidence_factors.append(0.95)  # High confidence in trend
                    
                    # Generate trend comment based on data
                    if trend_direction == 'upward' and pct_change > 5:
                        trend_comment = "The price trend appears positive with noteworthy growth."
                        recommendation = "Buy"
                    elif trend_direction == 'downward' and pct_change < -5:
                        trend_comment = "The price trend is negative with observable decline."
                        recommendation = "Sell"
                    else:
                        trend_comment = "The price trend is relatively stable or mixed."
                        recommendation = "Hold"
                    
                    insights.append(trend_comment)
                else:
                    # Trend data exists but is incomplete
                    analysis_components['trend_quality'] = 0.5
                    confidence_factors.append(0.5)
                    trend_comment = "Trend data is incomplete."
                    insights.append(trend_comment)
            else:
                # No trend data
                confidence_factors.append(0.0)
                trend_comment = "No trend data available."
                insights.append(trend_comment)
            
            # ========================================
            # 2. NEWS SENTIMENT ANALYSIS
            # ========================================
            news_comment = ""
            
            if news_list and len(news_list) > 0:
                analysis_components['has_news_data'] = True
                
                # Analyze news sentiment
                positive_keywords = ['beats', 'growth', 'rises', 'record', 'surge', 
                                     'profit', 'gains', 'strong', 'exceeds', 'soars']
                negative_keywords = ['misses', 'drops', 'falls', 'decline', 'weak', 
                                     'loss', 'plunge', 'disappoints', 'slumps', 'cuts']
                
                positive_news = []
                negative_news = []
                
                for n in news_list:
                    title_lower = n['title'].lower()
                    if any(word in title_lower for word in positive_keywords):
                        positive_news.append(n)
                    if any(word in title_lower for word in negative_keywords):
                        negative_news.append(n)
                
                # Calculate news quality score
                total_news = len(news_list)
                sentiment_news = len(positive_news) + len(negative_news)
                news_coverage = sentiment_news / total_news if total_news > 0 else 0
                
                analysis_components['news_quality'] = news_coverage
                
                # Confidence based on news quantity and sentiment clarity
                if total_news >= 5 and news_coverage > 0.6:
                    confidence_factors.append(0.90)  # Good news coverage
                elif total_news >= 3 and news_coverage > 0.4:
                    confidence_factors.append(0.70)  # Fair news coverage
                elif total_news >= 1:
                    confidence_factors.append(0.50)  # Limited news
                else:
                    confidence_factors.append(0.0)
                
                # Generate news comment and adjust recommendation
                if positive_news:
                    news_comment = f"Recent news headlines show positive sentiment ({len(positive_news)} positive signals)."
                    if recommendation == "Hold":
                        recommendation = "Buy"
                    elif recommendation == "Sell":
                        recommendation = "Hold"  # News improves outlook
                elif negative_news:
                    news_comment = f"Recent news headlines highlight concerns ({len(negative_news)} negative signals)."
                    if recommendation == "Buy":
                        recommendation = "Hold"  # News adds caution
                    elif recommendation == "Hold":
                        recommendation = "Sell"  # News confirms weakness
                else:
                    news_comment = "Recent news appears neutral."
                
                insights.append(news_comment)
            else:
                # No news data
                confidence_factors.append(0.3)  # Low confidence without news
                news_comment = "No recent news available for analysis."
                insights.append(news_comment)
            
            # ========================================
            # 3. RISK & STABILITY FROM VOLATILITY + ANOMALIES
            # ========================================
            # These fields come from the upgraded AnalysisAgent
            if trend_summary and isinstance(trend_summary, dict):
                stability = trend_summary.get('stability_label', "Unknown")
                anomaly_count = trend_summary.get('anomaly_count', 0)
            else:
                stability = "Unknown"
                anomaly_count = 0
            
            # Default risk factor
            risk_factor = 1.0
            
            if stability == "Highly Variable" or anomaly_count >= 3:
                risk_note = (
                    "⚠️ High volatility or several anomalies detected — treat this as a higher-risk situation."
                )
                # Soften overly aggressive recommendation
                if recommendation == "Buy":
                    recommendation = "Hold"
                # Higher risk → slightly lower confidence later
                risk_factor = 0.8
            elif stability == "Stable" and anomaly_count == 0:
                risk_note = "✅ Price behaviour is stable with no major anomalies detected."
                # Stable → slightly boost confidence later
                risk_factor = 1.1
            else:
                risk_note = "ℹ️ Some variability is present but within a normal range."
                risk_factor = 1.0
            
            insights.append(f"Risk & stability: {risk_note}")
            
            # ========================================
            # 4. CALCULATE OVERALL CONFIDENCE
            # ========================================
            # Base confidence from data quality
            if confidence_factors:
                base_confidence = sum(confidence_factors) / len(confidence_factors)
            else:
                base_confidence = 0.0
            
            # Adjust confidence based on completeness
            has_both = analysis_components['has_trend_data'] and analysis_components['has_news_data']
            completeness_factor = 1.0 if has_both else 0.7
            
            # Recommendation confidence (stronger signals = higher confidence)
            if recommendation == "Buy" or recommendation == "Sell":
                # Strong signal from clear trend or news
                analysis_components['recommendation_confidence'] = 0.85
            else:
                # Hold recommendation (less decisive)
                analysis_components['recommendation_confidence'] = 0.65
            
            # Final confidence calculation (before risk)
            final_confidence = (
                base_confidence * 0.6 +                                      # 60% weight on data quality
                completeness_factor * 0.2 +                                  # 20% weight on completeness
                analysis_components['recommendation_confidence'] * 0.2       # 20% weight on recommendation clarity
            )
            
            # 🔥 NEW: apply risk factor based on stability/anomalies
            final_confidence = final_confidence * risk_factor
            
            # Ensure confidence is between 0 and 1
            final_confidence = max(0.0, min(1.0, final_confidence))
            
            execution_time = time.time() - start_time
            
            # ========================================
            # 5. PREPARE INSIGHT SUMMARY
            # ========================================
            insight_summary = {
                'trend_summary': trend_summary,
                'news_summary': news_list,
                'insights': insights,
                'recommendation': recommendation,
                'confidence': final_confidence,  # ✅ Include confidence in output
                'analysis_quality': analysis_components,
                'stability_label': stability,
                'anomaly_count': anomaly_count
            }
            
            # ========================================
            # 6. EVALUATE WITH DYNAMIC CONFIDENCE
            # ========================================
            success = (
                analysis_components['has_trend_data'] or 
                analysis_components['has_news_data']
            )
            
            self.evaluator.evaluate(
                agent_name=self.name,
                success=success,
                confidence=final_confidence,  # ✅ Dynamic confidence!
                details={
                    'ticker': ticker,
                    'execution_time': execution_time,
                    'recommendation': recommendation,
                    'has_trend_data': analysis_components['has_trend_data'],
                    'has_news_data': analysis_components['has_news_data'],
                    'news_count': len(news_list) if news_list else 0,
                    'trend_quality': analysis_components['trend_quality'],
                    'news_quality': analysis_components['news_quality'],
                    'stability_label': stability,          # ✅ NEW
                    'anomaly_count': anomaly_count,        # ✅ NEW
                    'confidence_breakdown': {
                        'base_confidence': base_confidence,
                        'completeness_factor': completeness_factor,
                        'recommendation_confidence': analysis_components['recommendation_confidence'],
                        'risk_factor': risk_factor         # ✅ NEW
                    }
                }
            )
            
            print(f"✅ {self.name}: Insight generated in {execution_time:.2f}s")
            print(f"   Recommendation: {recommendation}")
            print(f"   Confidence: {final_confidence:.2f}")
            print(f"   Data Quality: Trend={analysis_components['trend_quality']:.2f}, News={analysis_components['news_quality']:.2f}")
            print(f"   Stability: {stability}, Anomalies: {anomaly_count}, Risk factor: {risk_factor:.2f}")
            
            return insight_summary
            
        except Exception as e:
            execution_time = time.time() - start_time
            
            # Evaluate failure
            self.evaluator.evaluate(
                agent_name=self.name,
                success=False,
                confidence=0.0,
                details={
                    'ticker': ticker,
                    'error': str(e),
                    'execution_time': execution_time
                }
            )
            
            print(f"❌ {self.name} failed: {e}")
            print(f"   Execution time: {execution_time:.2f}s")
            
            return {
                'trend_summary': trend_summary,
                'news_summary': news_list,
                'insights': ["Error generating insights"],
                'recommendation': "Hold",
                'confidence': 0.0,
                'error': str(e)
            }


# ============================================================================
# USAGE EXAMPLE WITH EVALUATION
# ============================================================================

# Create insight agent with evaluator (assumes `evaluator` and `trend_results` already exist)
insight_agent = InsightAgent(
    name="InsightAgent",
    model="gemini-2.5-flash-lite",
    instruction="You generate actionable stock insights using trend analysis and current news.",
    tools=['google_search'],
    output_key="stock_insight",
    evaluator=evaluator  # ✅ Pass evaluator
)

# Get news for the company
news_headlines = insight_agent.get_google_news(company="Apple")

# Generate insights with evaluation
insights_result = insight_agent.generate_insight(
    trend_summary=trend_results,
    news_list=news_headlines,
    ticker='AAPL'  # ✅ Added ticker for better tracking
)

# Print results
print("\n" + "="*70)
print("INSIGHTS RESULT:")
print("="*70)
print(f"Recommendation: {insights_result['recommendation']}")
print(f"Confidence: {insights_result['confidence']:.2f}")
print(f"\nInsights:")

for insight in insights_result['insights']:
    print(f"  - {insight}")

print("="*70)

# Show evaluation summary
evaluator.print_summary()


## REPORT AGENT

In [ ]:
# Define the ReportAgent class 

class ReportAgent:
    def __init__(self, name, model, instruction, output_key):
        self.name = name
        self.model = model
        self.instruction = instruction
        self.output_key = output_key

    def generate_trend_graph(self, timeseries_df, value_col='Close', ma_col='MA'):
        import matplotlib.pyplot as plt
        import base64
        from io import BytesIO

        plt.figure(figsize=(8, 4))
        plt.plot(timeseries_df['Date'], timeseries_df[value_col], label='Close Price', color='blue')
        if ma_col in timeseries_df.columns:
            plt.plot(timeseries_df['Date'], timeseries_df[ma_col], label='Moving Avg', color='red')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title('Price Trend with Moving Average')
        plt.legend()
        plt.tight_layout()

        buf = BytesIO()
        plt.savefig(buf, format="png")
        plt.close()
        buf.seek(0)
        img_str = base64.b64encode(buf.read()).decode("utf-8")
        return img_str

    def generate_html_report(self, insight_summary, company_name, timeseries_df):
        # === Pull core pieces from insight + trend ===
        img_base64 = self.generate_trend_graph(timeseries_df, value_col='Close', ma_col='MA')

        trend = insight_summary['trend_summary']
        recommendation = insight_summary.get('recommendation', 'N/A')
        confidence = insight_summary.get('confidence', 0.0)

        direction = trend.get('trend_direction', 'unknown')
        pct_change = trend.get('percentage_change', 0.0) or 0.0
        latest_value = trend.get('latest_value', 0.0)
        latest_ma = trend.get('latest_moving_average', 0.0)
        window = trend.get('moving_average_window', 20)

        # Volatility & risk fields from AnalysisAgent
        stability = trend.get('stability_label', 'Unknown')
        value_std = trend.get('value_std_dev', None)
        cv = trend.get('coefficient_of_variation', None)
        anomaly_count = trend.get('anomaly_count', 0)

        # Optional analysis quality from InsightAgent
        analysis_quality = insight_summary.get('analysis_quality', {})
        trend_quality = analysis_quality.get('trend_quality', 0.0)
        news_quality = analysis_quality.get('news_quality', 0.0)
        rec_conf = analysis_quality.get('recommendation_confidence', 0.0)

        # === Friendly text/format helpers ===
        arrow = "➖"
        if direction == "upward":
            arrow = "📈"
        elif direction == "downward":
            arrow = "📉"

        if recommendation == "Buy":
            rec_color = "#1b5e20"  # dark green
            rec_dot = "🟢"
        elif recommendation == "Sell":
            rec_color = "#b71c1c"  # dark red
            rec_dot = "🔴"
        else:
            rec_color = "#f57c00"  # orange
            rec_dot = "🟡"

        # Risk rating based on stability + anomalies + CV
        if stability == "Stable" and anomaly_count == 0 and (cv is not None and cv < 0.1):
            risk_rating = "Low"
            risk_tag = "🟢 Low"
        elif stability == "Highly Variable" or anomaly_count >= 3:
            risk_rating = "High"
            risk_tag = "🔴 High"
        else:
            risk_rating = "Medium"
            risk_tag = "🟡 Medium"

        value_std_text = f"{value_std:.2f}" if value_std is not None else "N/A"
        cv_text = f"{cv:.2f}" if cv is not None else "N/A"

         # === Executive summary (1–2 lines) ===
        if direction == "upward":
            trend_phrase = "strong upward price trend"
        elif direction == "downward":
            trend_phrase = "downward price pressure"
        else:
            trend_phrase = "mixed or sideways price movement"

        if confidence >= 0.8:
            conf_label = "high"
        elif confidence >= 0.6:
            conf_label = "moderate"
        else:
            conf_label = "low"

        exec_summary = (
            f"{company_name} currently shows a {trend_phrase} with "
            f"{stability.lower()} volatility and an overall <b>{risk_rating.lower()} risk</b> profile. "
            f"Based on price behaviour, volatility and recent news sentiment, "
            f"the system issues a <span style='color:{rec_color};'><b>{recommendation}</b></span> "
            f"recommendation with <b>{conf_label} confidence</b>."
        )

        exec_section = f"""
        <h2>Summary Overview</h2>
        <p>{exec_summary}</p>
        """


        # === Chart ===
        chart_section = f"""
        <img src="data:image/png;base64,{img_base64}" alt="Trend Graph" style="width:700px; border:1px solid #ccc; padding:4px;"/>
        """


        # === Trend overview ===
        trend_section = f"""
        <h2> Price Trend Overview</h2>
        <ul>
            <li><strong>Latest Value:</strong> {latest_value:.2f} </li>
            <li><strong>Moving Average ({window}-day):</strong> {latest_ma:.2f}</li>
            <li><strong>Trend Direction:</strong> {direction.title()} {arrow}</li>
            <li><strong>% Change (period):</strong> {pct_change:.2f}%</li>
        </ul>
        """

        # === Volatility & Risk ===
        volatility_section = f"""
        <h2>Volatility & Risk</h2>
        <ul>
            <li><strong>Stability:</strong> {stability}</li>
            <li><strong>Risk Rating:</strong> {risk_tag}</li>
            <li><strong>Price Std Dev:</strong> {value_std_text}</li>
            <li><strong>Coefficient of Variation:</strong> {cv_text}</li>
            <li><strong>Anomalies Detected:</strong> {anomaly_count}</li>
        </ul>
        <p style="font-size: 0.9em; color: #555;">
            Note: Anomalies are incorporated into the risk rating; raw anomaly rows are not shown
            to keep this summary concise and executive-friendly.
        </p>
        """

        # === News highlights ===
        news_items = insight_summary.get('news_summary', [])[:3]
        news_section = "<h2> Recent News Highlights</h2>"
        if news_items:
            news_section += "<ul>"
            for news in news_items:
                title = news.get('title', 'Untitled')
                link = news.get('link', '#')
                published = news.get('published', '')
                news_section += f"<li><a href='{link}' target='_blank'>{title}</a> {published}</li>"
            news_section += "</ul>"
        else:
            news_section += "<p>No recent news available.</p>"

        # === Key insights ===
        insights_section = "<h2>Key Insights</h2>"
        insights = insight_summary.get('insights', [])
        if insights:
            insights_section += "<ul>"
            for ins in insights:
                insights_section += f"<li>{ins}</li>"
            insights_section += "</ul>"
        else:
            insights_section += "<p>No additional insights available.</p>"

        # === Recommendation & Confidence breakdown ===
        breakdown_items = []
        breakdown_items.append(f"Trend signal quality: {trend_quality:.2f}")
        breakdown_items.append(f"News coverage quality: {news_quality:.2f}")
        breakdown_items.append(f"Recommendation clarity: {rec_conf:.2f}")

        breakdown_html = "<ul>"
        for item in breakdown_items:
            breakdown_html += f"<li>{item}</li>"
        breakdown_html += "</ul>"

        recommendation_section = f"""
        <h2>Recommendation & Confidence</h2>
        <p>
            <strong>Recommendation:</strong>
            <span style="color:{rec_color}; font-weight:bold;">{rec_dot} {recommendation}</span><br>
            <strong>Overall Confidence:</strong> {confidence:.2f}
        </p>
        <h3>Confidence Breakdown</h3>
        {breakdown_html}
        """
    

        # === Final HTML wrapper ===
        html_report = f"""
        <html>
        <head>
            <title>Executive Stock Report: {company_name}</title>
            <style>
                body {{
                    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Arial, sans-serif;
                    margin: 24px;
                    line-height: 1.6;
                    color: #222;
                }}
                h1 {{
                    font-size: 26px;
                    margin-bottom: 8px;
                }}
                h2 {{
                    font-size: 20px;
                    margin-top: 24px;
                    margin-bottom: 8px;
                }}
                h3 {{
                    font-size: 16px;
                    margin-top: 12px;
                    margin-bottom: 4px;
                }}
                ul {{
                    margin-top: 4px;
                    margin-bottom: 4px;
                }}
                a {{
                    color: #1565c0;
                    text-decoration: none;
                }}
                a:hover {{
                    text-decoration: underline;
                }}
            </style>
        </head>
        <body>
            <h1>Executive Stock Report: {company_name}</h1>
            {exec_section}
            {chart_section}
            {trend_section}
            {volatility_section}
            {news_section}
            {insights_section}
            {recommendation_section}
        </body>
        </html>
        """
        return html_report
        
# NOW instantiate before calling:
report_agent = ReportAgent(
    name="ReportAgent",
    model="gemini-2.5-flash-lite",
    instruction="Format insights into executive HTML report.",
    output_key="executive_report_html"
)

# Now, your call works:
html_report = report_agent.generate_html_report(
    insight_summary=insights_result,
    company_name="Apple",
    timeseries_df=timeseries_data
)

In [ ]:
# 1. Generate the HTML report
html_report = report_agent.generate_html_report(
    insight_summary=insights_result, 
    company_name="Apple", 
    timeseries_df=timeseries_data
)

# 2. Save the HTML string to a file
with open("Apple_Executive_Summary.html", "w", encoding="utf-8") as f:
    f.write(html_report)

print("Report saved as Apple_Executive_Summary.html")

# 3. (Optional) Open the file automatically (on Windows/Mac/Linux)
import webbrowser
webbrowser.open("Apple_Executive_Summary.html")

In [ ]:
from IPython.display import display, HTML

# Read and display the HTML in the notebook (make sure the path is correct!)
with open("Apple_Executive_Summary.html", "r", encoding="utf-8") as f:
    html_content = f.read()

display(HTML(html_content))